In [1]:
pip install spotipy

   ---------------------------------------- 0.0/255.6 kB ? eta -:--:--
   -------------------------------------- - 245.8/255.6 kB 7.6 MB/s eta 0:00:01
   ---------------------------------------- 255.6/255.6 kB 5.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

# Spotify API 인증 정보 설정 - Spotify wep api 로 접속 후 개인 토큰 키 입력
client_id = ''
client_secret = ''

auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)


In [56]:
# 플레이리스트 검색 함수
def search_playlists(query, limit=10):
    result = sp.search(q=query, type='playlist', limit=limit)
    playlists = result['playlists']['items']
    return playlists

In [57]:
# 플레이스트의 곡 가져오는 함수
def get_playlist_tracks(playlist_id):
    tracks = []
    results = sp.playlist_tracks(playlist_id)
    tracks.extend(results['items'])
    
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    
    return tracks

# 테스트코드
# 첫 번째 플레이리스트의 트랙 정보 가져오기
'''
if playlists:
    playlist_id = playlists[0]['id']
    tracks = get_playlist_tracks(playlist_id)

    # 각 트랙의 이름과 아티스트 출력
    for item in tracks:
        track = item['track']
        print(f"{track['name']} by {track['artists'][0]['name']}")
'''

'\nif playlists:\n    playlist_id = playlists[0][\'id\']\n    tracks = get_playlist_tracks(playlist_id)\n\n    # 각 트랙의 이름과 아티스트 출력\n    for item in tracks:\n        track = item[\'track\']\n        print(f"{track[\'name\']} by {track[\'artists\'][0][\'name\']}")\n'

In [58]:
# 곡의 audio_features 가져오는 함수
def get_all_audio_features(track_ids):
    audio_features = []
    
    # 최대 100개의 트랙 ID를 한 번에 요청할 수 있으므로 나누어서 요청
    for i in range(0, len(track_ids), 100):
        batch = track_ids[i:i+100]
        features = sp.audio_features(batch)
        audio_features.extend(features)
    
    return audio_features

# 테스트 코드
'''
# 트랙 ID 추출 및 오디오 특성 가져오기
track_ids = [track['track']['id'] for track in tracks]
audio_features = get_audio_features(track_ids)

# 첫 번째 트랙의 오디오 특성 출력
print(audio_features[0])
'''

"\n# 트랙 ID 추출 및 오디오 특성 가져오기\ntrack_ids = [track['track']['id'] for track in tracks]\naudio_features = get_audio_features(track_ids)\n\n# 첫 번째 트랙의 오디오 특성 출력\nprint(audio_features[0])\n"

In [59]:
# 트랙 정보 가져오는 함수
def get_track_data(tracks, audio_features):
    track_data = []
    
    # 오디오 특성을 track ID로 맵핑
    audio_features_dict = {af['id']: af for af in audio_features if af is not None}
    
    for item in tracks:
        track = item['track']
        track_id = track['id']
        
        # 트랙 정보 가져오기
        track_info = {
            #'Track ID': track_id,
            'Track Name': track['name'],
            'Album Name': track['album']['name'],
            'Artist(s)': ', '.join([artist['name'] for artist in track['artists']]),
            'Release Date': track['album']['release_date'],
            'Popularity': track['popularity'],
            'Duration (ms)': track['duration_ms'],
            'Explicit': track['explicit'],
            'Track URL': track['external_urls']['spotify'],
        }
        
        # 해당 트랙의 오디오 특성을 가져오기
        audio_features = audio_features_dict.get(track_id)
        if audio_features:
            track_info.update({
                'Danceability': audio_features['danceability'],
                'Energy': audio_features['energy'],
                'Key': audio_features['key'],
                'Loudness': audio_features['loudness'],
                'Mode': audio_features['mode'],
                'Speechiness': audio_features['speechiness'],
                'Acousticness': audio_features['acousticness'],
                'Instrumentalness': audio_features['instrumentalness'],
                'Liveness': audio_features['liveness'],
                'Valence': audio_features['valence'],
                'Tempo': audio_features['tempo'],
                'Duration (ms)': audio_features['duration_ms'],
                'Time Signature': audio_features['time_signature'],
                #'Track URI': audio_features['uri'],
                #'Track Href': audio_features['track_href'],
                #'Analysis URL': audio_features['analysis_url']
            })
        
        track_data.append(track_info)
    
    return track_data

In [60]:
# csv 파일로 저장하는 함수
def save_to_csv(data, filename):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)

### 한개의 검색어로만 파일 생성

In [61]:
# '2024년 인기차트' 키워드로 플레이리스트 검색
playlists = search_playlists('2024년 인기차트', limit=3)
track_list = []

for playlist in playlists:
    playlist_id = playlist['id']
    tracks = get_playlist_tracks(playlist_id)
    
    # 모든 트랙 ID 수집
    track_ids = [track['track']['id'] for track in tracks]
    
    # 한 번에 모든 오디오 특성을 수집
    audio_features = get_all_audio_features(track_ids)
    
    # 트랙 정보와 오디오 특성을 결합
    track_data = collect_full_track_data(tracks, audio_features)
    
    # track_data 리스트를 track_list에 확장하여 추가 (중첩 방지)
    track_list.extend(track_data)

# CSV 파일로 저장
save_to_csv(track_list, 'full_audio_features_and_track_info_2024_test.csv')
print("Track data saved to 'full_audio_features_and_track_info_2024_test.csv'")

Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache


Track data saved to 'full_audio_features_and_track_info_2024_test.csv'


### 각 키워드 다른 파일로 저장

In [62]:
# 검색할 키워드 리스트
search_keywords = ['멜론차트', '노동요', '분위기 팝송', '드라이브', '산책', '신나는', '운동']

for keyword in search_keywords:
    playlists = search_playlists(keyword, limit=3)
    track_list = []
    
    for playlist in playlists:
        playlist_id = playlist['id']
        tracks = get_playlist_tracks(playlist_id)
        
        # 모든 트랙 ID 수집
        track_ids = [track['track']['id'] for track in tracks]
        
        # 한 번에 모든 오디오 특성을 수집
        audio_features = get_all_audio_features(track_ids)
        
        # 트랙 정보와 오디오 특성을 결합
        track_data = collect_full_track_data(tracks, audio_features)
        
        # track_data 리스트를 track_list에 확장하여 추가 (중첩 방지)
        track_list.extend(track_data)
    
    # CSV 파일로 저장 (키워드에 따라 파일명 변경) playlist_keyword_info
    save_to_csv(track_list, f'playlist_keyword_info_{keyword}.csv')
    print(f"Track data for '{keyword}' saved to 'playlist_keyword_info_{keyword}.csv'")

Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache


Track data for '멜론차트' saved to 'full_audio_features_and_track_info_멜론차트.csv'


Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at:

Track data for '노동요' saved to 'full_audio_features_and_track_info_노동요.csv'


Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache


...

Track data for '산책' saved to 'full_audio_features_and_track_info_산책.csv'


Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache


Track data for '신나는' saved to 'full_audio_features_and_track_info_신나는.csv'


Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache


Track data for '운동' saved to 'full_audio_features_and_track_info_운동.csv'


### 키워드 한 개의 파일로 저장

In [63]:
# 검색할 키워드 리스트
search_keywords = ['멜론차트', '노동요', '분위기 팝송', '드라이브', '산책', '신나는', '운동']
all_track_data = []  # 모든 키워드의 트랙 데이터를 저장할 리스트

for keyword in search_keywords:
    playlists = search_playlists(keyword, limit=3)
    track_list = []
    
    for playlist in playlists:
        playlist_id = playlist['id']
        tracks = get_playlist_tracks(playlist_id)
        
        # 모든 트랙 ID 수집
        track_ids = [track['track']['id'] for track in tracks]
        
        # 한 번에 모든 오디오 특성을 수집
        audio_features = get_all_audio_features(track_ids)
        
        # 트랙 정보와 오디오 특성을 결합
        track_data = collect_full_track_data(tracks, audio_features)
        
        # 각 키워드에 해당하는 트랙 데이터를 추가
        track_list.extend(track_data)
    
    # 각 키워드에 대한 트랙 데이터를 전체 리스트에 추가
    all_track_data.extend(track_list)

# 모든 데이터를 하나의 CSV 파일로 저장
save_to_csv(all_track_data, 'playlist_info_all_keywords.csv')
print("All track data saved to 'playlist_info_all_keywords.csv'")

Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at:

All track data saved to 'full_audio_features_and_track_info_all_keywords.csv'
